# GRPA Coding

Goverment Perfromance and Results Act (GPRA) coding for intake data of recovery specialist for the Institure for Prevention and Recovery (IFPR) at RWJ Baranabas

In [1]:
#importing pandas
import pandas as pd
import numpy as np

In [2]:
# importing data and parsing datetime columns
df = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA_Coding_File.csv", parse_dates = ['ADMIT_DT','FORM_DTTM','FORM_UPDATE_DTTM','DOB'
])


#subsetting to include Peer Recovery Program episodes
df = df[df['EPISODE'] =='IFPR - Peer Recovery Program'].sort_values(by = ['FORM_DTTM','MRN']) # <- sorting df

In [3]:
#Renaming column to create ClientID
df.rename(columns = {'CSN':'ClientID'}, inplace = True)

In [4]:
#Inserting BatchID columns
df['BatchID'] = ''

In [5]:
#function for days columns that exceed 30 days
def exceed_thirty_days(x):
    if x > 30:
        return 30
    else:
        return x

## Creating GPRA 1

In [6]:
#Creating function to create column ClientDescripTreat
def Client_Descrip_Treat (x):
    if x == 'Treatment grant client':
        return 1
    else:
        return 0 
    
#Creating column ClientDescripTreat via applying function Client_Descrip_Treat
df['ClientDescripTreat'] = df['CLIENT_TYPE'].apply(lambda x: Client_Descrip_Treat (x))

In [7]:
#Creating function to create column ClientDescripRecov
def Client_Descrip_Recov (x):
    if x == 'Client in recovery grant':
        return 1
    else:
        return 0
    
#Creating column ClientDescripRecov via applying function Client_Descrip_Recov
df['ClientDescripRecov'] = df['CLIENT_TYPE'].apply(lambda x: Client_Descrip_Recov (x))

In [8]:
#Creating column GrantNo column
df['GrantNo'] = 'TI085280'

In [9]:
#Creating function for interview type
def interview_type_code(x):
    if x == 'Intake':
        return 1
    elif x == '6-month follow-up':
        return 2
    elif x == '3-month follow-up':
        return 4
    elif x == 'Discharge':
        return 5
    
#Creating new column via applying interview_type_code function to values in interview type column
df['InterviewType'] = df['INTERVIEW_TYPE'].apply(lambda x: interview_type_code(x))

In [10]:
#Creating function for conducted interview
def conducted_interview(x):
    if x == '6-month follow-up' or x == '3-month follow-up' or x == 'Discharge' or x == 'Intake':
        return 1
    else:
        return 0
    
# Creating a new column via applying the conducted_interview function to the values in the interview type column
df['ConductedInterview'] = df['INTERVIEW_TYPE'].apply(lambda x: conducted_interview(x))

In [11]:
df['INTERVIEW_TYPE']

1394               Intake
1387               Intake
1386    6-month follow-up
1385    6-month follow-up
1380               Intake
              ...        
1440               Intake
1451               Intake
1454               Intake
1438               Intake
1436               Intake
Name: INTERVIEW_TYPE, Length: 865, dtype: object

In [12]:
#Renaming column to InterviewDate and changing data type to output in the format MM/DD/YYYY
df['InterviewDate'] = df['FORM_DTTM'].dt.strftime('%m/%d/%Y')

def conducted_interview_code(row):
    if row['ConductedInterview'] ==1:
        return row['InterviewDate']
    else:
        return ''
    
df['InterviewDate'] = df.apply(conducted_interview_code,axis=1)

In [13]:
# Quality check
df['InterviewDate']

1394    07/13/2023
1387    01/01/2024
1386    01/01/2024
1385    01/02/2024
1380    01/02/2024
           ...    
1440    09/28/2024
1451    09/28/2024
1454    09/28/2024
1438    09/29/2024
1436    09/29/2024
Name: InterviewDate, Length: 865, dtype: object

In [14]:
# Rolling Count to create IntakeSeqNum column
df['IntakeSeqNum'] = df.groupby('MRN')['ADMIT_DT'].cumcount() + 1 # <- rolling count +1 to start count at 1

In [15]:
#Creating InactFlag Column
df['MRN_CNT'] = df.groupby(['PATIENT'])['MRN'].transform('count') # <- will return count of MRN the length of df
df['MAX_ADMIT_DT'] = df.groupby(['PATIENT'])['ADMIT_DT'].transform('max') # <- will return MAX ADMIT_DT the length of df

# Function across multiple columns
def inactflag (row):
    if row['MRN_CNT'] == 1:
        return 0
    elif row['MRN_CNT'] > 1 and row['ADMIT_DT'] == row['MAX_ADMIT_DT']:
        return 0
    else:
        return 1
df['InactFlag'] = df.apply(inactflag, axis =1 ) # <- axis 1 will apply function across columns 

In [16]:
df['InactFlag'].unique()

array([1, 0], dtype=int64)

In [17]:
# Assigning 0 for active grants
df['GrantInactFlag'] = 0

In [18]:
#Creating a function for Quarter
def year_qt (x):
    if x in [10,11,12]: # <- in kwarg that references a list 
        return 1
    elif x in [1,2,3]:
        return 2
    elif x in [4,5,6]:
        return 3
    elif x in [7,8,9]:
        return 4    

In [19]:
#Extracting year from ADMIT_DT to create column FFY
df['FFY'] = df['FORM_DTTM'].dt.year

#Extracting year from ADMIT_DT to create column Month
df['Month'] =  df['FORM_DTTM'].dt.month

#Extracting year from ADMIT_DT to create column Quarter
df['Quarter'] =  df['Month'].apply(lambda x: year_qt (x))

# Convering DOB to datetime format
df['DOB'] = pd.to_datetime(df['DOB'])

#Adding if then condition to account for InterviewType column 

#BirthMonth
def interview_type_dob (row): # <- function to create conidition based on interview type
    if row['InterviewType'] == 1: # <- intake
        return row['DOB'].month # <- returning month
    else: # <- return null value if not intake
        return ''
df['BirthMonth'] = df.apply(interview_type_dob, axis=1)


# BirthYear
def interview_type_year (row): # <- function to create conidition based on interview type
    if row['InterviewType'] == 1: # <- intake
        return row['DOB'].year # <- returning year 
    else: # <- returns null value if not intake
        return ''
df['BirthYear'] = df.apply(interview_type_year,axis =1)


#Age
def interview_type_age (row):
    if row['InterviewType'] == 1: # <- intake
        return row['FFY'] - row['BirthYear']
    else:
        return ''

df['Age'] = df.apply(interview_type_age, axis=1)


#AgeGroup
def age_group(row):# <- creating conditions for intake
    if row['Age'] >= 18 and row['Age'] < 25:
        return 1
    elif row['Age'] >= 25 and row['Age'] < 35:
        return 3
    elif row['Age'] >= 35 and row['Age'] < 45:
        return 4
    elif  row['Age'] >= 45 and row['Age'] < 55:
        return 5
    elif row['Age'] >=55 and row['Age'] < 65:
        return 6
    elif row['Age'] >= 65:
        return 7
    else:
        return -9


def interview_type_age_group (row):
    if row['InterviewType'] == 1:
        return age_group(row) # <- returning conditions for intakes via age_group function
    else: # return null values if not intake
        return ''
    
df['AgeGroup'] = df.apply(interview_type_age_group,axis=1)

In [20]:
#Creating function for gender code 
def gender_code(row):
    if row['GENDER'] == 'Male':
        return 1
    elif row['GENDER'] == 'Female':
        return 2
    else:
        return 7
    
def gender_type_code (row):
    if row['InterviewType'] == 1:
        return gender_code(row)
    else:
        return ''
    
df['Gender'] = df.apply(gender_type_code,axis=1)

#Renaming column to create column GenderSpec
df.rename(columns = {'WHAT_DO_YOU_CONSIDER_YOURSELF_TO_BE_OTHER':'GenderSpec'}, inplace = True)

In [21]:
#Creating a function for HispanicLatino Column
def Hispanic_Latino (row):
    if row['HISPANIC_LATIN_SPANISH_ORIGIN'] == 'Yes':
        return 1
    elif row['HISPANIC_LATIN_SPANISH_ORIGIN'] == 'No (Skip to Question 3)':
        return 0
    elif  row['HISPANIC_LATIN_SPANISH_ORIGIN'] == 'REFUSED (Skip to Question 3)':
        return -7
    else:
        return -9
    
def Hispanic_Latino_code (row):
    if row['InterviewType'] == 1:
        return Hispanic_Latino (row)
    else:
        return ''
    
df['HispanicLatino'] = df.apply(Hispanic_Latino_code,axis=1)

In [22]:
#Inputing 'NaN' for missing values in the WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF column
df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].fillna('NaN')

#Creating substrings
df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].str.split(',').str[0]

#Creating a function for column EthnicCentranAmerican
def Central_American(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Central American':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0
    
def Central_American_code (row):
    if row['InterviewType'] ==1:
        return Central_American(row)
    else:
        return ''

df['EthnicCentralAmerican'] = df.apply(Central_American_code,axis=1)

In [23]:
#Creating a function for column EthnicCuban
def Cuban(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Cuban':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0

def Cuban_code (row):
    if row['InterviewType'] ==1:
        return Cuban(row)
    else:
        return ''

df['EthnicCuban'] = df.apply(Cuban_code,axis=1)

In [24]:
#Creating a function for column EthnicDominican
def DR(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Dominican':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0

def DR_code(row):
    if row['InterviewType'] ==1:
        return DR(row)
    else:
        return ''

df['EthnicDominican'] = df.apply(DR_code,axis=1)

In [25]:
 #Creating a function for column EthnicMexican
def Mexican(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Mexican':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0

def Mexican_code(row):
    if row['InterviewType'] ==1:
        return Mexican(row)
    else:
        return ''

df['EthnicMexican'] = df.apply(Mexican_code,axis=1)

In [26]:
#Creating a function for column EthnicPuertoRican
def PR(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Puerto Rican':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0

def PR_code(row):
    if row['InterviewType'] ==1:
        return PR(row)
    else:
        return ''
    
df['EthnicPuertoRican'] =  df.apply(PR_code,axis=1)

In [27]:
#Creating a function for column EthnicSouthAmerican
def South_America(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'South American':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0
    
def South_America_code (row):
    if row['InterviewType'] ==1:
        return South_America (row)
    else:
        return ''

df['EthnicSouthAmerican'] = df.apply(South_America_code,axis=1)    

In [28]:
#Creating a function for column EthnicOther
def Ethnic_Other(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Other (SPECIFY)':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0

def Ethnic_Other_code(row):
    if row['InterviewType']==1:
        return Ethnic_Other(row)
    else:
        return ''
    

df['EthnicOther'] = df.apply(Ethnic_Other_code,axis=1)

In [29]:
#Renaming column
df.rename(columns = {'ETHNIC_GROUP_OTHER':'EthnicOtherSpec'}, inplace = True)

def EthnicOtherSpec_code (row):
    if row['InterviewType'] == 1:
        return row['EthnicOtherSpec']
    else:
        return ''
    
df['EthnicOtherSpec'] = df.apply(EthnicOtherSpec_code,axis=1)

In [30]:
#Fill null values with 'NaN' in the Race column
df['RACE'] = df['RACE'].fillna('NaN')

#Creating substrings 
df['RACE'] = df['RACE'].str.split(',').str[0]

#Creating function for column RaceBlack
def Race_Black(row):
    if row['RACE'] == 'Black or African American':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    elif row['RACE'] == 'Other (SPECIFY), REFUSED':
        return -7
    else:
        return 0

def Race_Black_code(row):
    if row['InterviewType'] ==1:
        return Race_Black(row)
    else:
        return ''
    
df['RaceBlack'] = df.apply(Race_Black_code,axis=1)

In [31]:
#Creating function for column RaceWhite
def Race_White(row):
    if row['RACE'] == 'White':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0

def Race_White_code(row):
    if row['InterviewType'] ==1:
        return Race_White(row)
    else:
        return ''

df['RaceWhite'] = df.apply(Race_White_code,axis=1)

In [32]:
#Creating function for column RaceAmericanIndian
def Race_Am_Indian(row):
    if row['RACE'] == 'American Indian':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
def Race_Am_Indian_code(row):
    if row['InterviewType']==1:
        return Race_Am_Indian(row)
    else:
        return ''
    
df['RaceAmericanIndian'] = df.apply(Race_Am_Indian_code,axis=1) 

In [33]:
#Creating function for column RaceAlaskaNative
def Race_Alaska_Native(row):
    if row['RACE'] == 'Alaska Native':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
def Race_Alaska_Native_code(row):
    if row['InterviewType']==1:
        return Race_Alaska_Native(row)
    else:

        return ''
    
df['RaceAlaskaNative'] = df.apply(Race_Alaska_Native_code,axis=1)

In [34]:
df['RaceAlaskaNative']

1394    0
1387    0
1386     
1385     
1380    0
       ..
1440    0
1451    0
1454    0
1438    0
1436    0
Name: RaceAlaskaNative, Length: 865, dtype: object

In [35]:
#Creating function for column RaceAsianIndian
def Race_Asian_Indian(row):
    if row['RACE']== 'Asian Indian':
        return 1
    elif row['RACE']== 'NaN':
        return -9
    elif row['RACE']== 'REFUSED':
        return -7
    else:
        return 0
    
def Race_Asian_Indian_code(row):
    if row['InterviewType'] ==1:
        return Race_Asian_Indian(row)
    else:
        return ''
    
df['RaceAsianIndian'] = df.apply(Race_Asian_Indian_code,axis=1) 

In [36]:
#Creating function for column RaceChinese
def Race_Chinese(row):
    if row['RACE'] == 'Chinese':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0

def Race_Chinese_code(row):
    if row['InterviewType'] ==1:
        return Race_Chinese(row)
    else:
        return ''
    
df['RaceChinese'] = df.apply(Race_Chinese_code,axis=1)

In [37]:
#Creating function for column RaceFilipino
def Race_Filipino(row):
    if row['RACE'] == 'Filipino':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
def Race_Filipino_code(row):
    if row['InterviewType']==1:
        return Race_Filipino(row)
    else:
        return ''

df['RaceFilipino'] = df.apply(Race_Filipino_code,axis=1)

In [38]:
#Creating function for column RaceJapanese
def Race_Japanese(row):
    if row['RACE'] == 'Japanese':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
def Race_Japanese_code(row):
    if row['InterviewType']==1:
        return Race_Japanese(row)
    else:
        return ''
    
df['RaceJapanese'] = df.apply(Race_Japanese_code,axis=1)

In [39]:
#Creating function for column RaceKorean
def Race_Korean(row):
    if row['RACE'] == 'Korean':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
def Race_Korean_code(row):
    if row['InterviewType']==1:
        return Race_Korean(row)
    else:
        return ''
    
df['RaceKorean'] = df.apply(Race_Korean_code,axis=1)

In [40]:
#Creating function for column RaceVietnamese
def Race_Vietnamese(row):
    if row['RACE'] == 'Vietnamese':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
def Race_Vietnamese_code(row):
    if row['InterviewType']== 1:
        return Race_Vietnamese(row)
    else:
        return ''
    
df['RaceVietnamese'] = df.apply(Race_Vietnamese,axis=1)

In [41]:
#Creating function for column RaceOtherAsian
def Race_Other_Asian(row):
    if row['RACE'] == 'Other Asian':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
def Race_Other_Asian_code(row):
    if row['InterviewType']==1:
        return Race_Other_Asian(row)
    else:
        return ''
    
df['RaceOtherAsian'] = df.apply(Race_Other_Asian_code,axis=1) 

In [42]:
#Creating function for column RaceNativeHawaiian
def Race_Hawaiian(row):
    if row['RACE'] == 'Hawaiian':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0

def Race_Hawaiian_code(row):
    if row['InterviewType']==1:
        return Race_Hawaiian(row)
    else:
        return ''
    
df['RaceNativeHawaiian'] = df.apply(Race_Hawaiian_code,axis=1)

In [43]:
#Creating function for column RaceGuamanianChamorro
def Race_Guamanian_Chamorro(row):
    if row['RACE'] == 'Guamanian Chamorro':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
def Race_Guamanian_Chamorro_code(row):
    if row['InterviewType']==1:
        return Race_Guamanian_Chamorro(row)
    else:
        return ''
    
df['RaceGuamanianChamorro'] = df.apply(Race_Guamanian_Chamorro_code,axis=1)

In [44]:
#Creating function for column RaceSamoan
def Race_Samoan(row):
    if row['RACE'] == 'Samoan':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
def Race_Samoan_code(row):
    if row['InterviewType']==1:
        return Race_Samoan(row)
    else:
        return ''

df['RaceSamoan'] = df.apply(Race_Samoan_code,axis=1)

In [45]:
#Creating function for column RaceOtherPacificIslander
def Race_Pacific_Islander(row):
    if row['RACE'] == 'Pacific Islander':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0

def Race_Pacific_Islander_code(row):
    if row['InterviewType']==1:
        return Race_Pacific_Islander(row)
    else:
        return ''

df['RaceOtherPacificIslander'] = df.apply(Race_Pacific_Islander_code,axis=1)

In [46]:
#Creating function for column RaceOther
def Race_Other(row):
    if row['RACE'] == 'Other (SPECIFY)':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
def Race_Other_code(row):
    if row['InterviewType']==1:
        return Race_Other(row)
    else:
        return ''

df['RaceOther'] = df.apply(Race_Other_code,axis=1)

In [47]:
#Renaming column 
df.rename(columns = {'RACE_OTHER':'RaceSpec'}, inplace = True)

#Renaming column to create column RaceSpec
df['RaceSpec'] = df['RaceSpec'].str.title()

def RaceSpec_code(row):
    if row['InterviewType']== 1:
        return row['RaceSpec']
    else:
        return ''
    
df['RaceSpec'] = df.apply(RaceSpec_code,axis=1)

In [48]:
# Filling missing in values in SPEAK_LANGUAGE_OTHER_THAN_ENGLISH column with 'NaN'
df['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] = df['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'].fillna('NaN')

#Creating function for LangNotEnglishAtHome
def Lang_Eng_Not_Spoken(row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return -1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'Yes':
        return 1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'No (Skip to Question 5)':
        return 0
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'REFUSED (Skip to Question 5)':
        return -7
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'NaN':
        return -9

def Lang_Eng_Not_Spoken_code_home(row):
    if row['InterviewType']==1:
        return Lang_Eng_Not_Spoken(row)
    else:
        return -1

df['LangNotEnglishAtHome'] = df.apply(Lang_Eng_Not_Spoken_code_home,axis=1)

#Creating function to create column LangNotEnglishSpoken
def Lang_Not_English_Spoken (row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return -1
    elif row['WHAT_OTHER_LANGUAGE'] == 'Spanish':
        return 1
    elif row['WHAT_OTHER_LANGUAGE'] == 'Other (SPECIFY)':
        return 2
    else:
        return -1
    
def Lang_Not_English_Spoken_code(row):
    if row['InterviewType']==1 and row['LangNotEnglishAtHome'] == 1:
        return Lang_Not_English_Spoken(row)
    else:
        return -1
    
df['LangNotEnglishSpoken'] = df.apply(Lang_Not_English_Spoken_code,axis=1)

In [49]:
#Renaming column to create column LangNotEnglishSpokenSpec
df.rename(columns = {'LANGUAGE_OTHER':'LangNotEnglishSpokenSpec'}, inplace = True)

In [50]:
#Creating LangNotSpanishSpokenSpec column
df['LangNotSpanishSpokenSpec'] = ''

In [51]:
#Creating function to create column LangNotSpanishAtHome
def Lang_Not_Spanish_Spoken_Home (row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'No (Skip to Question 5)':
        return -1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Yes' or row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Other (SPECIFY)':
        return 1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return 0
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'REFUSED (Skip to Question 5)':
        return -7
    else:
        return -1
    
def Lang_Not_Spanish_Spoken_Home_code(row):
    if row['LangNotEnglishSpoken'] == 2:
        return -1
    elif row['LangNotEnglishAtHome'] == 0:
        return -1
    elif row['InterviewType']==1:
        return Lang_Not_Spanish_Spoken_Home(row)
    else:
        return -1
    
df['LangNotSpanishAtHome'] = df.apply(Lang_Not_Spanish_Spoken_Home_code,axis=1)  



#Creating function to create column LangNotSpanishSpoken
def Lang_Not_Spanish_Spoken (row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'No (Skip to Question 5)':
        return -1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Yes':
        return 1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Other (SPECIFY)':
        return 2
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'REFUSED (Skip to Question 5)':
        return -7
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return -1
    else:
        return -1
    
def Lang_Not_Spanish_Spoken_code(row):
    if row['LangNotEnglishAtHome'] == 0:
        return -1
    elif row['LangNotEnglishSpoken'] == 2:
        return -1
    elif row['InterviewType']==1 and row['LangNotSpanishAtHome']==1:
        return Lang_Not_Spanish_Spoken(row)
    else:
        return -1
    
df['LangNotSpanishSpoken'] = df.apply(Lang_Not_Spanish_Spoken_code,axis=1) 


In [52]:
#filling missing values with 'NaN'
df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].fillna('NaN')

#Creating substrings
df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].str.split(',').str[0]

#Creating function for column SexIdentHeterosexual
def Straight (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Straight or Heterosexual':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0
    
def Straight_code(row):
    if row['InterviewType']==1:
        return Straight(row)
    else:
        return ''
    
df['SexIdentHeterosexual'] = df.apply(Straight_code,axis=1)

In [53]:
#Creating function for column SexIdentHomoSexual
def Homosexual (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Homosexual (Gay or Lesbian)':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0

def Homosexual_code(row):
    if row['InterviewType']==1:
        return Homosexual(row)
    else:
        return ''
    
df['SexIdentHomosexual'] = df.apply(Homosexual_code,axis=1)

In [54]:
#Creating function for column SexIdentBiSexual
def Bisexual (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Bisexual':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0
    
def Bisexual_code(row):
    if row['InterviewType']==1:
        return Bisexual(row)
    else:
        return ''
    
df['SexIdentBisexual'] = df.apply(Bisexual_code,axis=1)

In [55]:
#Creating function for column SexIdentQueerPanaQuest
def Queer_Pan_Quest (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Queer, Pansexual, And/Or Questioning':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0
    
def Queer_Pan_Quest_code(row):
    if row['InterviewType']==1:
        return Queer_Pan_Quest(row)
    else:
        return ''
    
df['SexIdentQueerPanaQuest'] = df.apply(Queer_Pan_Quest_code,axis=1)

In [56]:
#Creating function for column SexIdentAsexual
def Asexual (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Asexual':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0
    
def Asexual_code(row):
    if row['InterviewType']==1:
        return Asexual(row)
    else:
        return ''
    
df['SexIdentAsexual'] = df.apply(Asexual_code,axis=1)

In [57]:
df['SexIdentAsexual']

1394    0
1387    0
1386     
1385     
1380    0
       ..
1440    0
1451    0
1454    0
1438    0
1436    0
Name: SexIdentAsexual, Length: 865, dtype: object

In [58]:
#Creating function for column SexIdentOther
def Other (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Other':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0
    
def Other_code(row):
    if row['InterviewType']==1:
        return Other(row)
    else:
        return ''

    
df['SexIdentOther'] = df.apply(Other_code,axis=1)

In [59]:
#Creating function for column SexIdentSpec
def Other_Spec (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Other (SPECIFY)':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0
    
def Other_Spec_code(row):
    if row['InterviewType']==1 and row['SexIdentOther'] == 1:
        return Other_Spec(row)
    else:
        return ''
    
df['SexIdentSpec'] = df.apply(Other_Spec_code,axis=1)  

In [60]:
#Filling in null values with'NaN'
df['RELATIONSHIP_STATUS'] = df['RELATIONSHIP_STATUS'].fillna('NaN')

#Creating function for RelationshipStatus column
def Relationship_Status (row):
    if row['RELATIONSHIP_STATUS'] == 'Married':
        return 1
    elif row['RELATIONSHIP_STATUS'] == 'Single':
        return 2
    elif row['RELATIONSHIP_STATUS'] == 'Divorced':
        return 3
    elif row['RELATIONSHIP_STATUS'] == 'Separated':
        return 4
    elif row['RELATIONSHIP_STATUS'] == 'Widowed':
        return 5
    elif row['RELATIONSHIP_STATUS'] == 'In a relationship':
        return 6
    elif row['RELATIONSHIP_STATUS'] == 'In multiple relationships':
        return 7
    elif row['RELATIONSHIP_STATUS'] == 'REFUSED':
        return -7
    elif row['RELATIONSHIP_STATUS'] == 'NaN':
        return -9

def Relationship_Status_code(row):
    if row['InterviewType']==1:
        return Relationship_Status (row)
    else:
        return ''

df['RelationshipStatus'] = df.apply(Relationship_Status_code,axis=1)


In [61]:
#Filling missing values with 'NaN'
df['CURRENTLY_PREGNANT'] = df['CURRENTLY_PREGNANT'].fillna('NaN')

#Creating function for Pregnant column
def Pregnant (row):
    if row['CURRENTLY_PREGNANT'] == 'Yes':
        return 1
    elif row['CURRENTLY_PREGNANT'] == 'No':
        return 0
    elif row['CURRENTLY_PREGNANT'] == 'Do not know':
        return -8
    elif row['CURRENTLY_PREGNANT'] == 'REFUSED':
        return -7
    elif row['CURRENTLY_PREGNANT'] == 'NaN':
        return -9
    
def Pregnant_code(row):
    if row['InterviewType']==1:
        return Pregnant(row)
    else:
        return ''

df['Pregnant'] = df.apply(Pregnant_code,axis=1)

In [62]:
#Replacing null values with 'NaN'
df['HAVE_CHILDREN'] = df['HAVE_CHILDREN'].fillna(-9)

# Creating function to create new column Children
def Child (row):
    if row['HAVE_CHILDREN'] == 'Yes':
        return 1
    elif row['HAVE_CHILDREN'] == 'No (Skip to Question 9)':
        return 0
    elif row['HAVE_CHILDREN'] == 'REFUSED (Skip to Question 9)':
        return -7
    elif row['HAVE_CHILDREN'] == -9:
        return -9
    
def Child_code(row):
    if row['InterviewType']==1:
        return Child(row)
    else:
        pass
    
df['Children'] = df.apply(Child_code,axis=1)

In [63]:
#Renaming columns 
df.rename(columns = {'CHILDREN_UNDER_18':'ChildrenUnder18Nr'}, inplace = True)


def ChildrenUnder18Nr(row):
    if row['Children'] == 1:
        return row['ChildrenUnder18Nr']
    else:
        pass
    
df['ChildrenUnder18Nr'] = df.apply(ChildrenUnder18Nr,axis=1)

In [65]:
#Creating function to create column ChildrenCustody
def custody(row):
    if row['CHILDREN_UNDER_18_LIVING_SOMEWHERE_ELSE_COURT_INTERVENTION'] == 'Yes':
        return 1
    elif  row['CHILDREN_UNDER_18_LIVING_SOMEWHERE_ELSE_COURT_INTERVENTION'] == 'No (Skip to Question 9)':
        return 0

def Children_Custody_code(row):
    if row['Children'] != 1:
        pass
    elif row['ChildrenUnder18Nr'] <= 0:
        pass
    elif row['ChildrenUnder18Nr'] > 0:
        return custody(row)
    
df['ChildrenCustody'] = df.apply(Children_Custody_code,axis=1)

In [66]:
#Renaming column 
df.rename(columns = {'CHILDREN_REMOVED_FROM_CLIENTS_CARE':'ChildrenCustodyNr'}, inplace = True)

def custody_nr (row):
    if row['ChildrenCustody'] == 1:
        return row['ChildrenCustodyNr'] 

def ChildrenCustodyNr(row):
    if row['Children'] != 1:
        pass
    elif row['ChildrenUnder18Nr'] <= 0:
        pass
    else:
        return custody_nr(row)

df['ChildrenCustodyNr'] = df.apply(ChildrenCustodyNr,axis=1)

In [71]:
#Creating function to create column ChildrenReunited
def child_reunited(row):
    if row ['REUNITED_WITH_CHILDREN_UNDER_18_PREV_REMOVED_FROM_CARE'] == 'Yes':
        return 1
    elif row ['REUNITED_WITH_CHILDREN_UNDER_18_PREV_REMOVED_FROM_CARE']== 'No':
        return 0

def Children_Reunited_code(row):
    if row['Children'] != 1:
        pass
    elif row['ChildrenUnder18Nr'] <= 0:
        pass
    elif row['ChildrenCustody'] != 1:
        pass
    else:
        return child_reunited(row)
    
df['ChildrenReunited'] = df.apply(Children_Reunited_code,axis=1)

In [72]:
#Renaming column 
df.rename(columns = {'CHILDREN_REUNITED_WITH_CLIENT':'ChildrenReunitedNr'},inplace = True)

def ChildrenReunitedNr(row):
    if row['ChildrenReunited'] == 1:
        return row['ChildrenReunitedNr']
    
df['ChildrenReunitedNr'] = df.apply(ChildrenReunitedNr,axis=1)

In [73]:
#Replacing null values 'NaN'
df['EVER_SERVED_IN_MILITARY'] = df['EVER_SERVED_IN_MILITARY'].fillna('NaN')

#Creating function to create new column MilitaryServed
def Military_Served (row):
    if row['EVER_SERVED_IN_MILITARY'] == 'No':
        return 0
    elif row['EVER_SERVED_IN_MILITARY'] == 'Yes, In the Armed Forces':
        return 1
    elif row['EVER_SERVED_IN_MILITARY'] == 'Yes, In the National Guard':
        return 3
    elif row['EVER_SERVED_IN_MILITARY'] == 'REFUSED':
        return -7
    elif row['EVER_SERVED_IN_MILITARY'] == 'NaN':
        return -9
    
def Military_Served_code(row):
    if row['InterviewType']==1:
        return Military_Served(row)
    else:
        return ''
    
df['MilitaryServed'] = df.apply(Military_Served_code,axis=1)

In [74]:
#Replacing null values 'NaN'
df['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] = df['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'].fillna('NaN')

#Creating function to create new column ServicesTravelTime
def Travel_Time (row):
    if row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'Half an hour or less':
        return 1
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'Between half an hour and one hour':
        return 2
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'Between one hour and one and a half hours':
        return 3
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'Between one and a half hours and two hours':
        return 4
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'Two hours or more':
        return 5
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'REFUSED':
        return -7
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'NaN':
        return -9
    
def Travel_Time_code(row):
    if row['InterviewType']==1:
        return Travel_Time(row)
    else:
        return ''

    
df['ServicesTravelTime'] = df.apply(Travel_Time_code,axis=1)

In [75]:
#Renaming column
df.rename(columns = {'ALCOHOL_NUMBER_OF_DAYS':'AlcoholDays'}, inplace = True)

#Replacing null values with 0 to create column AlcoholDays
df['AlcoholDays'] = df['AlcoholDays'].fillna(0)

#Replacing values greater than 30 with 30
df['AlcoholDays'] = df['AlcoholDays'].apply(lambda x:exceed_thirty_days(x))

In [76]:
#Creating function for AlcoholRoute
def Alcohol_Route (row):
    if row['ALCOHOL_ROUTE'] == '1. Oral':
        return 1
    elif row['AlcoholDays'] ==0:
        return -1
    else:
        return -1
    
#Creating column AlcoholRoute via applying the function Alcohol_Route
df['AlcoholRoute'] = df.apply(Alcohol_Route, axis = 1 )

In [77]:
#Creating column AlcoholOther1Days 
df['AlcoholOther1Days'] = 0

#Creating column AlcoholOther1Route 
df['AlcoholOther1Route'] = -1

df['AlcoholOther1Spec'] = " "

#Replacing missing values to create respective columns
df['AlcoholOther2Days'] = 0
df['AlcoholOther2Route'] = -1
df['AlcoholOther2Spec'] = " "
df['AlcoholOther3Days'] = 0
df['AlcoholOther3Route'] = -1
df['AlcoholOther3Spec'] = " "

In [78]:
#Renaming colulmn
df.rename(columns = {'HEROIN_NUMBER_OF_DAYS':'OpioidsHeroinDays'}, inplace = True)

#Replacing null values with 0 to create column OpioidsHeroinDays
df['OpioidsHeroinDays'] = df['OpioidsHeroinDays'].fillna(0)

df['OpioidsHeroinDays'] = df['OpioidsHeroinDays'].apply(lambda x:exceed_thirty_days(x))

In [90]:
#Creating substrings
df['HEROIN_ROUTE'] = df['HEROIN_ROUTE'].str.split(',').str[0]

#Creating function for column OpioidsHeroinRoute
def Heroin_Route (row):
    if row['HEROIN_ROUTE'] == '1. Oral':
        return 1
    elif row['HEROIN_ROUTE'] == '2. Intranasal':
        return 2
    elif row['HEROIN_ROUTE'] == '3. Vaping':
        return 6
    elif row['HEROIN_ROUTE'] == '5. Non-IV Injection':
        return 4
    elif row['HEROIN_ROUTE'] == '6. Intravenous (IV) Injection':
        return 5
    elif row['HEROIN_ROUTE'] == '0. Other':
        return 0
    elif row['HEROIN_ROUTE'] == '4. Smoking':
        return 3
    elif row['OpioidsHeroinDays'] == 0:
        return -1
    else:
        return -1
    
#Creating column OpioidsHeroinRoute via applying function Heroin_Route
df['OpioidsHeroinRoute'] = df.apply(Heroin_Route, axis = 1)

In [80]:
#Renaming columns
df.rename(columns = {'MORPHINE_NUMBER_OF_DAYS':'OpioidsMorphineDays'}, inplace = True)

# Amending 'MORPHINE_NUMBER_OF_DAYS' column via replacing null values with 0 to create OpioidsMorphineDays column
df['OpioidsMorphineDays'] = df['OpioidsMorphineDays'].fillna(0)

df['OpioidsMorphineDays'] = df['OpioidsMorphineDays'].apply(lambda x: exceed_thirty_days(x))

In [81]:
#Creating function to create new column OpioidsMorphineRoute
def Morphine_Route (row):
    if row['MORPHINE_ROUTE'] == '1. Oral':
        return 1
    elif row['MORPHINE_ROUTE'] == '6. Intravenous (IV) Injection':
        return 2
    elif row['MORPHINE_ROUTE'] == '0. Other':
        return 0
    elif row['OpioidsMorphineDays'] == 0:
        return -1
    
#Creating new column OpioidsMorphineRoute via applying the function Morphine_Route
df['OpioidsMorphineRoute'] = df.apply(Morphine_Route, axis = 1)

In [82]:
#Renaming columns 
df.rename(columns = {'FENTANYL_NUMBER_OF_DAYS':'OpioidsFentanylDays'},inplace = True)

# Replacing null values with 0 to create OpioidsFentanylDays column 
df['OpioidsFentanylDays'] = df['OpioidsFentanylDays'].fillna(0)

df['OpioidsFentanylDays'] = df['OpioidsFentanylDays'].apply(lambda x:exceed_thirty_days(x))

In [83]:
#Creating function to create new column OpioidsFentanylRoute
def Fentanyl_Route (row):
    if row['FENTANYL_ROUTE'] == '1. Oral':
        return 1
    elif row['FENTANYL_ROUTE'] == '2. Intranasal':
        return 2
    elif row['FENTANYL_ROUTE'] == '6. Intravenous (IV) Injection':
        return 5
    elif row['FENTANYL_ROUTE'] == '0. Other':
        return 0
    elif row['OpioidsFentanylDays']==0:
        return -1
    else:
        return -1
    
#Creating new column OpioidsFentanylRoute via applying the function Fentanyl_Route
df['OpioidsFentanylRoute'] = df.apply(Fentanyl_Route, axis = 1)

In [84]:
#Renaming column
df.rename(columns = {'DILAUDID_NUMBER_OF_DAYS':'OpioidsDilaudidDays'}, inplace = True)

#Replacing null values with 0 to create values for column OpioidsDilaudidDays
df['OpioidsDilaudidDays'] = df['OpioidsDilaudidDays'].fillna(0)

df['OpioidsDilaudidDays'] = df['OpioidsDilaudidDays'].apply(lambda x:exceed_thirty_days(x))

In [85]:
#Creating function for creating column OpioidsDilaudidRoute
def Dilaudid_Route (row):
    if row ['DILAUDID_ROUTE'] == '1. Oral':
        return 1
    elif row['OpioidsDilaudidDays'] == 0:
        return -1
    else:
        return -1

# Creating new column OpioidsDilaudidRoutev via applying function Dilaudid_Route  
df['OpioidsDilaudidRoute'] = df.apply(Dilaudid_Route,axis = 1)

In [86]:
#Renaming column
df.rename(columns = {'DEMEROL_NUMBER_OF_DAYS':'OpioidsDemerolDays'}, inplace = True)

#Replacing nulll values with 0 to create values for column OpioidsDemerolDays
df['OpioidsDemerolDays'] = df['OpioidsDemerolDays'].fillna(0)

df['OpioidsDemerolDays'] = df['OpioidsDemerolDays'].apply(lambda x: exceed_thirty_days(x))

## Exporting GPRA MAT-PDOA 1


In [87]:
# #Create variable by indexing columns and creating df
# GPRA1 = df[['MRN','ClientID','ClientDescripTreat','ClientDescripRecov','GrantNo',	'InterviewType',	'ConductedInterview',	'InterviewDate',	'IntakeSeqNum',	'GrantInactFlag',	'InactFlag',	'FFY',	'Quarter',	'Month',	'BirthMonth',	'BirthYear',	'Age',	'AgeGroup',	'Gender',	'GenderSpec',	'HispanicLatino',	'EthnicCentralAmerican',	'EthnicCuban',	'EthnicDominican',	'EthnicMexican',	'EthnicPuertoRican',	'EthnicSouthAmerican',	'EthnicOther',	'EthnicOtherSpec',	'RaceBlack',	'RaceWhite',	'RaceAmericanIndian',	'RaceAlaskaNative',	'RaceAsianIndian',	'RaceChinese',	'RaceFilipino',	'RaceJapanese',	'RaceKorean',	'RaceVietnamese',	'RaceOtherAsian',	'RaceNativeHawaiian',	'RaceGuamanianChamorro',	'RaceSamoan',	'RaceOtherPacificIslander',	'RaceOther',	'RaceSpec',	'LangNotEnglishAtHome',	'LangNotEnglishSpoken',	'LangNotEnglishSpokenSpec',	'LangNotSpanishAtHome',	'LangNotSpanishSpoken',	'LangNotSpanishSpokenSpec',	'SexIdentHeterosexual',	'SexIdentHomosexual',	'SexIdentBisexual',	'SexIdentQueerPanaQuest',	'SexIdentAsexual',	'SexIdentOther',	'SexIdentSpec',	'RelationshipStatus',	'Pregnant',	'Children',	'ChildrenUnder18Nr',	'ChildrenCustody',	'ChildrenCustodyNr',	'ChildrenReunited',	'ChildrenReunitedNr',	'MilitaryServed',	'ServicesTravelTime',	'AlcoholDays',	'AlcoholRoute',	'AlcoholOther1Days',	'AlcoholOther1Route',	'AlcoholOther1Spec',	'AlcoholOther2Days',	'AlcoholOther2Route',	'AlcoholOther2Spec',	'AlcoholOther3Days',	'AlcoholOther3Route',	'AlcoholOther3Spec',	'OpioidsHeroinDays',	'OpioidsHeroinRoute',	'OpioidsMorphineDays',	'OpioidsMorphineRoute',	'OpioidsFentanylDays',	'OpioidsFentanylRoute',	'OpioidsDilaudidDays',	'OpioidsDilaudidRoute',	'OpioidsDemerolDays']]

# #Exporting GPRA 1
# GPRA1.to_csv('GPRA1.csv', index = False, header = True, encoding = 'utf-8')

#Create variable by indexing columns and creating df
GPRA1_MAT_PDOA = df[['BatchID','PATIENT','PROVIDER_COUNTY','MRN','ClientID','ClientDescripTreat','ClientDescripRecov','GrantNo',	'InterviewType',	'ConductedInterview',	'InterviewDate','FORM_DTTM',	'IntakeSeqNum',	'GrantInactFlag',	'InactFlag',	'FFY',	'Quarter',	'Month',	'BirthMonth',	'BirthYear',	'Age',	'AgeGroup',	'Gender',	'GenderSpec',	'HispanicLatino',	'EthnicCentralAmerican',	'EthnicCuban',	'EthnicDominican',	'EthnicMexican',	'EthnicPuertoRican',	'EthnicSouthAmerican',	'EthnicOther',	'EthnicOtherSpec',	'RaceBlack',	'RaceWhite',	'RaceAmericanIndian',	'RaceAlaskaNative',	'RaceAsianIndian',	'RaceChinese',	'RaceFilipino',	'RaceJapanese',	'RaceKorean',	'RaceVietnamese',	'RaceOtherAsian',	'RaceNativeHawaiian',	'RaceGuamanianChamorro',	'RaceSamoan',	'RaceOtherPacificIslander',	'RaceOther',	'RaceSpec',	'LangNotEnglishAtHome',	'LangNotEnglishSpoken',	'LangNotEnglishSpokenSpec',	'LangNotSpanishAtHome',	'LangNotSpanishSpoken',	'LangNotSpanishSpokenSpec',	'SexIdentHeterosexual',	'SexIdentHomosexual',	'SexIdentBisexual',	'SexIdentQueerPanaQuest',	'SexIdentAsexual',	'SexIdentOther',	'SexIdentSpec',	'RelationshipStatus',	'Pregnant',	'Children',	'ChildrenUnder18Nr',	'ChildrenCustody',	'ChildrenCustodyNr',	'ChildrenReunited',	'ChildrenReunitedNr',	'MilitaryServed',	'ServicesTravelTime',	'AlcoholDays',	'AlcoholRoute',	'AlcoholOther1Days',	'AlcoholOther1Route',	'AlcoholOther1Spec',	'AlcoholOther2Days',	'AlcoholOther2Route',	'AlcoholOther2Spec',	'AlcoholOther3Days',	'AlcoholOther3Route',	'AlcoholOther3Spec',	'OpioidsHeroinDays',	'OpioidsHeroinRoute',	'OpioidsMorphineDays',	'OpioidsMorphineRoute',	'OpioidsFentanylDays',	'OpioidsFentanylRoute',	'OpioidsDilaudidDays',	'OpioidsDilaudidRoute',	'OpioidsDemerolDays']]

#Exporting GPRA 1
GPRA1_MAT_PDOA.to_csv('GPRA1_MAT_PDOA.csv', index = False, header = True, encoding = 'utf-8')